# 🧠 Data Processing & Planning — NYC Yellow Taxi Dataset (2019–2020)
### _Portfolio Project — Holberton School Machine Learning Track_
#### Author: **Alvi Sulaj**

This notebook outlines the full data-processing plan for the NYC Yellow Taxi dataset, covering:
- data sources  
- formats  
- exploratory analysis strategy  
- hypotheses  
- handling missing values & outliers  
- feature engineering  
- dataset splitting  
- storage structure  

This notebook serves as **Task 0: Planning**.

## 🔍 1. Data Sources

The primary dataset comes from the **NYC Taxi & Limousine Commission (TLC)**.

Official dataset link:  
🔗 https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

The dataset contains:
- pickup & dropoff timestamps  
- pickup & dropoff location IDs  
- trip distance  
- passenger count  
- fare details (fare, tips, tolls, total)  
- payment type  
- vendor information  

These files are released monthly and contain millions of rows each.

### 📦 Additional (Optional) Data Sources
To enrich the analysis:
- **NOAA Weather Data** → correlate weather with taxi demand  
- **NYC Borough Shapefiles** → spatial mapping  
- **Holiday Calendar** → detect seasonal spikes  

In [3]:
# Listing all available TLC monthly files (example)
import requests
from bs4 import BeautifulSoup

url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

links = [a['href'] for a in soup.find_all('a', href=True) if "yellow" in a['href']]
links[:10]  # show first 10

['/site/tlc/businesses/yellow-cab.page',
 '/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-07.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-08.parquet']

> 🔎 Note:
> In a real project, we could automatically list all TLC CSV files using web-scraping tools.
> However, the Holberton environment does not support `BeautifulSoup` or external installations.
> For this project, we will reference the monthly Yellow Taxi CSV files manually.

## 📄 2. Current Data Format

### Raw Format
- Multiple **CSV** files  
- Monthly structure (e.g., `yellow_tripdata_2019-01.csv`)
- Files are large (1M–7M rows per month)

### Working Format
- Loaded into **Pandas DataFrames**

### Preferred Export Format
- **Parquet** → efficient storage, compressed, faster loading  
- **Feather** (optional)
- **CSV** (fallback compatibility)

### Reason for Choosing Parquet
- 3–6x faster read/write  
- preserves schema  
- optimised for columnar analytics

In [1]:
import pandas as pd

# Example structure for loading one monthly CSV (placeholder)
sample_path = "/content/yellow_tripdata_2019-01.csv"   # You will replace with your path

try:
    df_sample = pd.read_csv(sample_path, nrows=5000)
    df_sample.head()
except FileNotFoundError:
    print("Sample file not found — this is expected in planning stage.")

Sample file not found — this is expected in planning stage.


## 🧬 3. Existing Features in Dataset

The dataset contains the following key fields:

### 🚕 Trip Metadata
- `tpep_pickup_datetime`
- `tpep_dropoff_datetime`
- `trip_distance`
- `passenger_count`

### 📍 Spatial Features
- `PULocationID` → pickup zone  
- `DOLocationID` → dropoff zone  

### 💵 Economic Features
- `fare_amount`
- `tip_amount`
- `tolls_amount`
- `mta_tax`
- `total_amount`

### 💳 Payment Information
- `payment_type`  
- `ratecodeID`  
- `vendorID`

These features will be deeply explored in EDA.

In [2]:
# Simulated structure to show how summary would be performed once data is loaded
try:
    df_sample.info()
except:
    print("Data not loaded yet — placeholder cell.")

Data not loaded yet — placeholder cell.


## 📊 4. Planned Exploratory Data Analysis (EDA)

A structured EDA will analyze:

### ✔ Missing Values
- Identify incomplete rows  
- Detect columns with high sparsity  
- Evaluate logical consistency  

### ✔ Distributions
- Trip distance distribution  
- Fare distribution  
- Duration distribution  

### ✔ Outliers
- Very short/long rides  
- Negative or impossible values  
- Abnormal fare-per-mile  

### ✔ Temporal Behaviors
- Trips by hour of day  
- Weekday vs weekend  
- Monthly seasonality  

### ✔ Geospatial Patterns
- Heatmaps of pickup zones  
- Flow analysis between boroughs  

## 🧪 5. Hypotheses and How They Will Be Tested

### 🟠 Hypothesis 1  
**Short trips have a higher fare-per-mile than long trips.**  
→ Test via grouping + comparing distributions.

### 🟠 Hypothesis 2  
**Taxi demand peaks during rush hours (8–10 AM, 4–7 PM).**  
→ Test via time-series counts.

### 🟠 Hypothesis 3  
**Rain increases taxi usage.**  
→ Merge with weather data + compare daily totals.

### 🟠 Hypothesis 4  
**Tip percentage increases at night.**  
→ Extract hour + analyze tip rates by time block.

## ⚠️ 6. Missing Data & Outlier Strategy

### Missing Data
- Drop rows missing timestamps  
- Impute categorical fields if patterns allow  
- Handle fare missingness with domain rules  

### Outliers
- Remove rides with:
  - distance == 0  
  - duration <= 1 minute  
  - fare <= 0  
  - speed > 120 mph  

### Outlier detection techniques:
- Z-score threshold  
- IQR (Interquartile Range)  
- Logical domain constraints  

## 🛠️ 7. Feature Engineering Plan

New features will be created to improve model performance:

### Time-based features
- pickup_hour  
- pickup_weekday  
- is_weekend  
- is_rush_hour  

### Trip dynamics
- trip_duration (minutes)  
- speed (miles per hour)  
- fare_per_mile  
- tip_percentage  

### Spatial enhancements
- borough_pickup  
- borough_dropoff  
- route_category (PU-DO combination)

## 🧩 8. Dataset Splitting Strategy

### Random Split
- 70% training  
- 15% validation  
- 15% testing  

### Time-Series Split (preferred for this dataset)
- Training → 2019  
- Validation → Jan–Jun 2020  
- Test → Jul–Dec 2020  

## 🗂️ 9. Storage Structure

project/
├── raw_data/
├── processed_data/
├── notebooks/
├── src/
├── env/
└── README.md


### Cleaned datasets will be saved as:
- `processed_data/cleaned_2019_2020.parquet`


---

# ✅ This completes the extended technical data plan.
